In [1]:
import torch
import pandas as pd
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler

In [2]:
if torch.cuda.is_available():
    device = 'cuda:0'
else:
    device = 'cpu'

In [3]:
class VtMLP(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(5, 64),
      nn.ReLU(),
      nn.Linear(64, 32),
      nn.ReLU(),
      nn.Linear(32, 1)
    )


  def forward(self, x):
    '''
      Forward pass
    '''
    return self.layers(x)

vt_model = VtMLP()
vt_model.load_state_dict(torch.load('../checkpoint/vt_mlp_model.pt'))
vt_model.eval() # we pretrained this model... so we'll use this model for inference

VtMLP(
  (layers): Sequential(
    (0): Linear(in_features=5, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [4]:
class FullWithVtDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, scale_data=True):
        if not torch.is_tensor(X) and not torch.is_tensor(y):
            if scale_data:
                vt = vt_model(torch.Tensor(X)).detach().numpy()
                X = np.concatenate([X, vt], 1)
                self.scaler_x = StandardScaler().fit(X)
                self.scaler_y = StandardScaler().fit(y)
                X = self.scaler_x.transform(X)
                y = self.scaler_y.transform(y)
            self.X = torch.from_numpy(X)
            self.y = torch.from_numpy(y)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, i):
        return self.X[i], self.y[i]   

In [5]:
full_df = pd.read_csv('../data_full/full_dataset.csv')
len(full_df)

86832

In [6]:
full_X = full_df[['W','L','T','Vgs','Vds']].to_numpy()
print(len(full_X))
full_y = full_df[['Ids']].to_numpy()
print(len(full_y))
dataset = FullWithVtDataset(full_X, full_y)
print(len(dataset))

86832
86832
86832


In [7]:
train_set, test_set = torch.utils.data.random_split(dataset, [69466,17366])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True, num_workers=1)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=False, num_workers=1)

In [8]:
class FullWithVtMLP(nn.Module):
    '''
    Multilayer Perceptron for regression.
    '''
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
        nn.Linear(6, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, 1)
        )


    def forward(self, x):
        '''
        Forward pass
        '''
        return self.layers(x)

mlp = FullWithVtMLP().to(device)
mlp.load_state_dict(torch.load('../checkpoint/full_vt_mlp_model_nrmse.pt'))
mlp.eval()

FullWithVtMLP(
  (layers): Sequential(
    (0): Linear(in_features=6, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [17]:
def NRMSELoss(y_hat, y, eps=1e-8):
    log_abs_y_hat = torch.log(torch.abs(y_hat))
    log_abs_y = torch.log(torch.abs(y))
    log_abs_delta = log_abs_y_hat - log_abs_y
    delta = y_hat - y
    len_delta = y_hat.size(dim=0)
    print(len_delta)
    return torch.sqrt((1/len_delta)*torch.sum((log_abs_delta/(log_abs_y+eps))**2)) + torch.sqrt((1/len_delta)*torch.sum((delta/(y+eps))**2))

In [22]:
predictions = torch.tensor([], dtype=torch.float).to(device)
actual = torch.tensor([], dtype=torch.float).to(device)
with torch.no_grad():
    mlp.eval()
    for data in test_loader:
        inputs, values = data
        inputs, values = inputs.float().to(device), values.float().to(device)
        values = values.reshape((values.shape[0], 1))

        outputs = mlp(inputs)
        # outputs = dataset.scaler_y.inverse_transform(outputs)
        predictions = torch.cat((predictions, outputs), 0)
        # values = dataset.scaler_y.inverse_transform(values)
        actual = torch.cat((actual, values), 0)
    
predictions = predictions.cpu().numpy()
actual = actual.cpu().numpy()

predictions = dataset.scaler_y.inverse_transform(predictions)
actual = dataset.scaler_y.inverse_transform(actual)

nrmse = NRMSELoss(torch.tensor(predictions), torch.tensor(actual))

print(predictions)
print(actual)

print(nrmse)

17366
[[-3.02987019e-06]
 [ 2.65838316e-04]
 [ 5.03392982e-07]
 ...
 [ 1.31149845e-05]
 [-3.71779743e-06]
 [ 9.45247120e-06]]
[[1.3121132e-11]
 [2.8809998e-04]
 [4.2224963e-11]
 ...
 [8.0840964e-06]
 [1.3121132e-11]
 [1.3121132e-11]]
tensor(960.5919)


In [12]:
960.5919/17366*100

5.531451687204883